In [55]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm.session import sessionmaker
from bs4 import BeautifulSoup
import cookielib
import urllib2
import requests
import re

In [56]:
def get_soup(url):
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html.decode('utf-8', 'ignore'), 'html.parser')
    results = soup.find_all('div', attrs = {'class': 'info'})
    print response.status_code
    return results

In [64]:
results = get_soup('http://www.theonion.com/')

200


In [104]:
def extract_features(results):
     # initialize lists
    titles = []
    dates = []
    links = []
    bodies = []
    sections = []
    
    for x in results:
        # get link
        link_suffix = x.find('a')['href']
        match = re.search('^/article(?![\w\d])', link_suffix)
        if match:
            link = 'http://www.theonion.com' + link_suffix
            links.append(link)
            
    
    # resoup it
    for link in links:
        response = requests.get(link)
        if response.status_code != 200:
            print response.status_code
        html = response.content
        new_soup = BeautifulSoup(html.decode('utf-8', 'ignore'), 'html.parser')
        
        # get title
        title = new_soup.find('title')
        titles.append(title)
        
        # get body
        body = new_soup.find('div', attrs = {'class': 'content-text'})
        paragraph = body.find('p').text.strip()
        p = paragraph.decode('utf-8', errors = 'optional')
        bodies.append(paragraph)
        
    data_dict = {
        'title': titles, 'link': links, 'body': bodies,
        'section': sections, 'date': dates
    }
    return data_dict

In [105]:
my_dict = extract_features(results)

UnicodeEncodeError: 'ascii' codec can't encode character u'\u2014' in position 10: ordinal not in range(128)

In [91]:
my_dict['body']

[u'WASHINGTON\u2014Praising the federal appellate judge\u2019s commitment to upholding the Constitution, President Trump hailed his Supreme Court justice nominee Neil Gorsuch as a fierce protector of the future amendment that will allow the president to temporarily suspend the right to assemble. \u201cI am confident that Judge Gorsuch will do everything in his power to see that every line of the Constitution, including the forthcoming amendment to grant the executive branch the power to halt all public protests, will be safeguarded as the law of the land,\u201d said Trump, adding that Gorsuch\u2019s strict adherence to the country\u2019s supreme law, both in its present and very near future iterations, was what drew him to the 49-year-old judge in the first place. \u201cIf his track record is any indication, Judge Gorsuch will act as a stalwart defender of our nation\u2019s founding legal document, which will soon contain a 28th amendment granting the president sole authority to restri